Decision Tree 圖解:

https://medium.com/jameslearningnote/%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90-%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%AC%AC3-5%E8%AC%9B-%E6%B1%BA%E7%AD%96%E6%A8%B9-decision-tree-%E4%BB%A5%E5%8F%8A%E9%9A%A8%E6%A9%9F%E6%A3%AE%E6%9E%97-random-forest-%E4%BB%8B%E7%B4%B9-7079b0ddfbda

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from sklearn import datasets

import matplotlib.pyplot as plt
import seaborn as sns

import random

global divding_line
dividing_line = '\n==================================================\n'

In [2]:
%matplotlib inline

In [3]:
iris_dataset = datasets.load_iris()

x = pd.DataFrame(iris_dataset['data'], columns = iris_dataset['feature_names'])
y = pd.DataFrame(iris_dataset['target'], columns = ['target'])

y = y.replace([0, 1, 2], ['Iris-setosar', 'Iris-versicolor', 'Iris-virginica'])

x = x.rename(columns = {'sepal length (cm)' : 'sepal_length(cm)', 'sepal width (cm)' : 'sepal_width(cm)'
                   , 'petal length (cm)' : 'petal_length(cm)', 'petal width (cm)' : 'petal_width(cm)'})

df = pd.concat([x, y], axis = 1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sepal_length(cm)  150 non-null    float64
 1   sepal_width(cm)   150 non-null    float64
 2   petal_length(cm)  150 non-null    float64
 3   petal_width(cm)   150 non-null    float64
 4   target            150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
df.head(10)

,sepal_length(cm),sepal_width(cm),petal_length(cm),petal_width(cm),target
0,5.1,3.5,1.4,0.2,Iris-setosar
1,4.9,3.0,1.4,0.2,Iris-setosar
2,4.7,3.2,1.3,0.2,Iris-setosar
3,4.6,3.1,1.5,0.2,Iris-setosar
4,5.0,3.6,1.4,0.2,Iris-setosar
5,5.4,3.9,1.7,0.4,Iris-setosar
6,4.6,3.4,1.4,0.3,Iris-setosar
7,5.0,3.4,1.5,0.2,Iris-setosar
8,4.4,2.9,1.4,0.2,Iris-setosar
9,4.9,3.1,1.5,0.1,Iris-setosar


In [5]:
df.isnull().sum()

sepal_length(cm)    0
sepal_width(cm)     0
petal_length(cm)    0
petal_width(cm)     0
target              0
dtype: int64

## Helper Functions

In [6]:
def split_data_to_train_test_set(dataFrame, portion_to_split = 0.2):
    indices = dataFrame.index.tolist()
    indices_test_dataset = random.sample(population = indices, k = int(portion_to_split * len(dataFrame)))
    
    test_dataset = dataFrame.iloc[indices_test_dataset]
    train_dataset = dataFrame.drop(indices_test_dataset, axis = 0)
    
    return train_dataset, test_dataset
    

In [7]:
random.seed(0)
train_df, test_df = split_data_to_train_test_set(df, portion_to_split = 0.2)

print('Training dataset with len: {}, testing with len: {}'.format(len(train_df), len(test_df)))

Training dataset with len: 120, testing with len: 30


In [8]:
def check_purity(array):
    labels = array[:, -1]
    unique_classes = np.unique(labels)
    
    if len(unique_classes) == 1:
        return True
    else:
        return False
    

In [9]:
print(train_df.values[70:90])

check_purity(train_df.values[70:90])

[[5.5 2.5 4.0 1.3 'Iris-versicolor']
 [5.5 2.6 4.4 1.2 'Iris-versicolor']
 [5.8 2.6 4.0 1.2 'Iris-versicolor']
 [5.0 2.3 3.3 1.0 'Iris-versicolor']
 [5.6 2.7 4.2 1.3 'Iris-versicolor']
 [5.7 3.0 4.2 1.2 'Iris-versicolor']
 [5.7 2.9 4.2 1.3 'Iris-versicolor']
 [6.2 2.9 4.3 1.3 'Iris-versicolor']
 [5.7 2.8 4.1 1.3 'Iris-versicolor']
 [6.3 3.3 6.0 2.5 'Iris-virginica']
 [5.8 2.7 5.1 1.9 'Iris-virginica']
 [7.1 3.0 5.9 2.1 'Iris-virginica']
 [6.5 3.0 5.8 2.2 'Iris-virginica']
 [7.6 3.0 6.6 2.1 'Iris-virginica']
 [4.9 2.5 4.5 1.7 'Iris-virginica']
 [6.7 2.5 5.8 1.8 'Iris-virginica']
 [7.2 3.6 6.1 2.5 'Iris-virginica']
 [6.5 3.2 5.1 2.0 'Iris-virginica']
 [6.4 2.7 5.3 1.9 'Iris-virginica']
 [6.8 3.0 5.5 2.1 'Iris-virginica']]


False

In [10]:
np.unique(train_df.values[:, -1], return_counts = True)

(array(['Iris-setosar', 'Iris-versicolor', 'Iris-virginica'], dtype=object),
 array([41, 38, 41], dtype=int64))

In [11]:
def classify(array):
    labels = array[:, -1]
    unique_classes, counts_unique_classes = np.unique(labels, return_counts = True)
    
    index_max_counts = counts_unique_classes.argmax()
    classification = unique_classes[index_max_counts]
    
    return classification

In [12]:
classify(train_df.values)

'Iris-setosar'

In [88]:
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [14]:
def split_dataset(array, split_column, split_threshold):
    split_column_values = array[:, split_column]
    
    type_of_feature = FEATURE_TYPES[split_column]
    
    if type_of_feature == 'continuous':
        left_splited_data = array[split_column_values <= split_threshold]
        right_splited_data = array[split_column_values > split_threshold]
    else:
        left_splited_data = array[split_column_values == split_threshold].copy()
        right_splited_data = array[split_column_values != split_threshold].copy()
    
    return left_splited_data, right_splited_data

## 不純度函數 --> 用來代表資訊量
### 1. Entropy
<img src="https://miro.medium.com/max/875/1*_otZEQ9iVAJirblwdVnIIw.png" />

### 2. Gini Impurity
<img src="https://miro.medium.com/max/1225/1*WXHY6siVegJRKcFqxCRSYQ.png" />

In [15]:
def calculate_entropy(array):
    labels = array[:, -1]
    
    _, counts = np.unique(labels, return_counts = True)
    
    probabilities = counts / np.sum(counts)
    
    entropy = np.sum(-probabilities * np.log2(probabilities))
    
    return entropy

In [16]:
def calculate_gini_impurity(array):
    labels = array[:, -1]
    
    _, counts = np.unique(labels, return_counts = True)
    
    probabilities = counts / np.sum(counts)
    
    gini_impurity = 1 - np.sum(probabilities**2)
    
    return gini_impurity

In [17]:
print(calculate_entropy(train_df.values))

1.5840530131956925


In [18]:
print(calculate_gini_impurity(train_df.values))

0.66625


In [19]:
np.unique(train_df.values[:, -1], return_counts = True)

(array(['Iris-setosar', 'Iris-versicolor', 'Iris-virginica'], dtype=object),
 array([41, 38, 41], dtype=int64))

## 信息增益 --> 用來挑選特徵
<img src="https://miro.medium.com/max/875/1*FjGUfcYt_Vyupv1KfNm_pA.png" />

In [20]:
def calculate_information_gain(criterion, left_splited_data, right_splited_data):
    num_total_data = len(left_splited_data) + len(right_splited_data)
    
    prob_left_splited_data = len(left_splited_data) / num_total_data
    prob_right_splited_data = len(right_splited_data) / num_total_data
    
    if criterion == 'entropy':
        information_gain = (prob_left_splited_data * calculate_entropy(left_splited_data)) + (prob_right_splited_data * calculate_entropy(right_splited_data))
    else:
        information_gain = (prob_left_splited_data * calculate_gini_impurity(left_splited_data)) + (prob_right_splited_data * calculate_gini_impurity(right_splited_data))
    
    return information_gain

In [21]:
print(calculate_information_gain('entropy', train_df.values[:50], train_df.values[50:]))

0.8542715202043386


In [22]:
print(calculate_information_gain('gini', train_df.values[:50], train_df.values[50:]))

0.4060952380952381


In [23]:
def pick_the_best_split(criterion, array, potential_splits):
    min_information_gain = 9999999

    for column_index, splits in potential_splits.items():
        for threshold in splits:
            left_splited_data, right_splited_data = split_dataset(array, column_index, threshold)
            
            if criterion == 'entropy':
                current_information_gain = calculate_information_gain('entropy', left_splited_data, right_splited_data)
            else:
                current_information_gain = calculate_information_gain('gini', left_splited_data, right_splited_data)
            
            if current_information_gain <= min_information_gain:
                min_information_gain = current_information_gain
                best_split_column = column_index
                best_threshold = threshold
    
    return best_split_column, best_threshold
    

In [39]:
pick_the_best_split('entropy', train_df.values, get_potential_splits(train_df.values))

(3, 0.8)

In [40]:
pick_the_best_split('gini', train_df.values, get_potential_splits(train_df.values))

(3, 0.8)

## Determine the type of features

In [41]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_threshold = 15
    
    for feature in df.columns:
        if feature != 'target':
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            
            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_threshold):
                feature_types.append('categorical')
            else:
                feature_types.append('continuous')
    
    return feature_types

## Decision Tree Algorithm

In [42]:
def determine_types_of_features(dataframe):
    
    feature_types = []
    n_unique_values_treshold = 15
    
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [89]:
def execute_decision_tree_alogrithm(criterion, dataFrame, counter = 0, min_samples = 2, max_depth = 5):
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = dataFrame.columns
        FEATURE_TYPES = determine_type_of_feature(dataFrame)
        data = dataFrame.values
        
    else:
        data = dataFrame
    
    if (check_purity(data) or (len(data) < min_samples) or (counter == max_depth)):
        classification = classify(data)
        return classification
    
    else:
        counter += 1

        potential_splits = get_potential_splits(data)
        best_split_column, best_threshold = pick_the_best_split(criterion, data, potential_splits)
        left_splited_data, right_splited_data = split_dataset(data, best_split_column, best_threshold)
        
        # determine the question for the decision tree node
        feature_name = COLUMN_HEADERS[best_split_column]
        type_of_feature = FEATURE_TYPES[best_split_column]
        
        if type_of_feature == "continuous":
            split_strategy = "{} <= {}".format(feature_name, best_threshold)
        else:
            split_strategy = "{} = {}".format(feature_name, best_threshold)
        
        
        root = {split_strategy : []}
        
        left_sub_tree = execute_decision_tree_alogrithm(criterion, left_splited_data, counter, min_samples, max_depth)
        right_sub_tree = execute_decision_tree_alogrithm(criterion, right_splited_data, counter, min_samples, max_depth)
        
        root[split_strategy].append(left_sub_tree)
        root[split_strategy].append(right_sub_tree)
        
        return root

In [90]:
tree = execute_decision_tree_alogrithm('entropy', train_df, max_depth = 3)
pprint(tree)

{'petal_width(cm) <= 0.8': ['Iris-setosar',
                            {'petal_width(cm) <= 1.65': [{'petal_length(cm) <= 4.95': ['Iris-versicolor',
                                                                                       'Iris-virginica']},
                                                         {'petal_length(cm) <= 4.85': ['Iris-virginica',
                                                                                       'Iris-virginica']}]}]}


In [91]:
tree = execute_decision_tree_alogrithm('gini', train_df, max_depth = 3)
pprint(tree)

{'petal_width(cm) <= 0.8': ['Iris-setosar',
                            {'petal_width(cm) <= 1.65': [{'petal_length(cm) <= 4.95': ['Iris-versicolor',
                                                                                       'Iris-virginica']},
                                                         {'petal_length(cm) <= 4.85': ['Iris-virginica',
                                                                                       'Iris-virginica']}]}]}


## Classification

In [92]:
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [93]:
example = test_df.iloc[0]
print(example)

sepal_length(cm)                5.1
sepal_width(cm)                 2.5
petal_length(cm)                  3
petal_width(cm)                 1.1
target              Iris-versicolor
Name: 98, dtype: object


In [111]:
def classify_dataFrame(dataFrame, tree):
    decision = list(tree.keys())[0]
    feature_name, comparison_operator, threshold = decision.split()
    
    if comparison_operator == '<=':
        if dataFrame[feature_name] <= float(threshold):
            answer = tree[decision][0]
        else:
            answer = tree[decision][1]
    else:
        if str(dataFrame[feature_name]) == threshold:
            answer = tree[decision][0]
        else:
            answer = tree[decision][1]
    
    if not isinstance(answer, dict):
        return answer
    
    else:
        residual_tree = answer
        return classify_dataFrame(dataFrame, residual_tree)

In [95]:
classify_dataFrame(example, tree)

'Iris-versicolor'

In [96]:
def calculate_accuracy(dataFrame, tree):
    dataFrame_copied = dataFrame.copy()
    dataFrame_copied['classification'] = dataFrame_copied.apply(classify_dataFrame, axis = 1, args = (tree, ))
    dataFrame_copied['classification_correct'] = dataFrame_copied['classification'].eq(dataFrame_copied['target'])
    
    accuaracy = dataFrame_copied['classification_correct'].mean()
    
    return accuaracy

In [97]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.9666666666666667

# Titanic Dataset

In [98]:
df_titanic = pd.read_csv("./data/Titanic.csv")
df_titanic['target'] = df_titanic.Survived
df_titanic = df_titanic.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis = 1)

In [99]:
df_titanic.isnull().sum(axis = 0)

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
target        0
dtype: int64

In [100]:
median_age = df_titanic.Age.median()
mode_embarked = df_titanic.Embarked.mode()[0]

df_titanic = df_titanic.fillna({'Age': median_age, 'Embarked' : mode_embarked})

# df_titanic['Embarked'] = df_titanic['Embarked'].map({'Q' : 1, 'S' : 2, 'C' : 3})

In [101]:
determine_type_of_feature(df_titanic)

['categorical',
 'categorical',
 'continuous',
 'categorical',
 'categorical',
 'continuous',
 'categorical']

In [102]:
left_data, right_data = split_dataset(df_titanic.values, split_column = 1, split_threshold = 'female')

In [103]:
left_data

array([[1, 'female', 38.0, ..., 71.2833, 'C', 1],
       [3, 'female', 26.0, ..., 7.925, 'S', 1],
       [1, 'female', 35.0, ..., 53.1, 'S', 1],
       ...,
       [3, 'female', 39.0, ..., 29.125, 'Q', 0],
       [1, 'female', 19.0, ..., 30.0, 'S', 1],
       [3, 'female', 28.0, ..., 23.45, 'S', 0]], dtype=object)

In [104]:
right_data

array([[3, 'male', 22.0, ..., 7.25, 'S', 0],
       [3, 'male', 35.0, ..., 8.05, 'S', 0],
       [3, 'male', 28.0, ..., 8.4583, 'Q', 0],
       ...,
       [2, 'male', 27.0, ..., 13.0, 'S', 0],
       [1, 'male', 26.0, ..., 30.0, 'C', 1],
       [3, 'male', 32.0, ..., 7.75, 'Q', 0]], dtype=object)

In [123]:
random.seed(0)

df_titanic_train, df_titanic_test = split_data_to_train_test_set(df_titanic, 0.2)
tree = execute_decision_tree_alogrithm('gini', df_titanic_train, max_depth = 5)
accuracy = calculate_accuracy(df_titanic_test, tree)

pprint(tree, width = 10)
accuracy

{'Sex = male': [{'Age <= 6.5': [{'SibSp = 4': [{'Fare <= 31.331249999999997': [0,
                                                                               {'Fare <= 35.5375': [1,
                                                                                                    0]}]},
                                               {'SibSp = 5': [0,
                                                              {'SibSp = 3': [0,
                                                                             1]}]}]},
                                {'Fare <= 26.26875': [{'Age <= 13.5': [{'Embarked = S': [1,
                                                                                         0]},
                                                                       {'Age <= 34.25': [0,
                                                                                         0]}]},
                                                      {'Fare <= 26.46875': [1,
                        

0.7921348314606742